In [ ]:
!pip install --upgrade pip -q

# System dependencies for PDF & OCR
!apt-get update -qq
!apt-get install -y -qq tesseract-ocr libtesseract-dev poppler-utils

# Python packages
!pip install --quiet \
  transformers sentence-transformers faiss-cpu yfinance pdfplumber PyMuPDF \
  python-dotenv tabulate pillow pytesseract streamlit pyngrok groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.2 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Selecting previously unselected package libleptonica-dev.
Preparing to unpack .../libleptonica-dev_1.82.0-3build1_amd64.deb ...
Unpacking libleptonica-dev (1.82.0-3build1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack .../libtesseract-dev_4.1.1-2.1build1_amd64.deb ...
Unpacking libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Selecting previously unselected package poppler-utils.
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, pickle, torch

# 🛠️ CHANGE THESE PATHS TO MATCH YOUR DRIVE
INPUT_DIR  = "/content/drive/MyDrive/FinancialNER_Input"
OUTPUT_DIR = "/content/drive/MyDrive/FinancialNER_Output"

os.makedirs(OUTPUT_DIR, exist_ok=True)

PKL_PATH  = f"{INPUT_DIR}/ner_json.pkl"     # your trained NER .pkl
PDF_PATH  = f"{INPUT_DIR}/apple.pdf"        # just for quick testing (optional)

print("PKL exists:", os.path.exists(PKL_PATH))
print("Sample PDF exists:", os.path.exists(PDF_PATH))
print("Output dir:", OUTPUT_DIR)


Mounted at /content/drive
PKL exists: True
Sample PDF exists: True
Output dir: /content/drive/MyDrive/FinancialNER_Output


In [ ]:
import torch, pickle
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

def safe_load_pkl(path):
    """Load a .pkl that may contain either a dict or a pipeline."""
    try:
        print("Trying torch.load(..., map_location='cpu')")
        obj = torch.load(path, map_location='cpu')
        return obj
    except Exception as e1:
        print("torch.load failed:", e1)
        try:
            print("Trying pickle.load(...)")
            with open(path, "rb") as f:
                obj = pickle.load(f)
            return obj
        except Exception as e2:
            raise RuntimeError(f"Both torch.load and pickle.load failed: {e1} | {e2}")

loaded = safe_load_pkl(PKL_PATH)
print("Loaded type:", type(loaded))

def build_ner_pipeline(loaded_obj):
    """Build a transformers pipeline from a loaded object."""
    # CASE 1: it is already a transformers pipeline
    if hasattr(loaded_obj, "__call__") and hasattr(loaded_obj, "task"):
        print("Detected: existing transformers pipeline")
        return loaded_obj

    # CASE 2: dict with state dict + label maps (your case from earlier)
    if isinstance(loaded_obj, dict) and "model_state_dict" in loaded_obj:
        print("Detected: state_dict with id2label/label2id")
        id2label = loaded_obj.get("id2label")
        label2id = loaded_obj.get("label2id")
        base_model_name = "dslim/bert-base-NER"  # can change if needed

        tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = AutoModelForTokenClassification.from_pretrained(
            base_model_name,
            num_labels=len(id2label),
            id2label=id2label,
            label2id=label2id
        )
        model.load_state_dict(loaded_obj["model_state_dict"], strict=False)
        ner_pipe = pipeline(
            "ner",
            model=model,
            tokenizer=tokenizer,
            aggregation_strategy="simple",
            device=-1  # CPU in Colab; change to 0 if GPU
        )
        return ner_pipe

    raise ValueError("Unsupported PKL format. Please inspect 'loaded' manually.")

ner_pipeline = build_ner_pipeline(loaded)
print("✅ NER pipeline ready.")

# save for the Streamlit app to load
with open(f"{OUTPUT_DIR}/ner_pipeline.pkl", "wb") as f:
    pickle.dump(ner_pipeline, f)
print("Saved pipeline to:", f"{OUTPUT_DIR}/ner_pipeline.pkl")


Trying torch.load(..., map_location='cpu')


/usr/local/lib/python3.12/dist-packages/torch/_weights_only_unpickler.py:549: UserWarning: Detected pickle protocol 4 in the checkpoint, which was not the default pickle protocol used by `torch.load` (2). The weights_only Unpickler might not support all instructions implemented by this protocol, please file an issue for adding support if you encounter this.
  warnings.warn(


torch.load failed: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 149

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.
Trying pickle.load(...)
Loaded type: <class 'dict'>
Detected: state_dict with id2label/label2id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


✅ NER pipeline ready.
Saved pipeline to: /content/drive/MyDrive/FinancialNER_Output/ner_pipeline.pkl


In [ ]:
import fitz, pdfplumber, pytesseract, re, io
from PIL import Image
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

EMB_MODEL_NAME = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL_NAME)

def extract_pages(pdf_bytes: bytes):
    """Return list of dicts: {'page': int, 'text': str} with OCR for empty pages."""
    pages = []
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        for i, page in enumerate(doc):
            txt = page.get_text("text") or ""
            if len(txt.strip()) < 30:
                pix = page.get_pixmap(dpi=150)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                txt = pytesseract.image_to_string(img)
            pages.append({"page": i+1, "text": txt})
    return pages

def chunk_text_from_pages(pages, max_chars=800, overlap=200):
    """Simple sliding window chunking over concatenated pages."""
    chunks = []
    meta = []
    for p in pages:
        text = re.sub(r"\s+", " ", p["text"]).strip()
        start = 0
        while start < len(text):
            end = min(len(text), start + max_chars)
            chunk = text[start:end]
            chunks.append(chunk)
            meta.append({"page": p["page"]})
            start = end - overlap
            if start < 0:
                start = 0
            if end == len(text):
                break
    return chunks, meta

def build_faiss_index(chunks):
    """Embed chunks and build FAISS L2 index."""
    if not chunks:
        return None, None
    print(f"Embedding {len(chunks)} chunks with {EMB_MODEL_NAME} ...")
    vectors = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors.astype(np.float32))
    return index, vectors

def search_chunks(query, chunks, index, k=5):
    """Return top-k (chunk, score) pairs."""
    if index is None:
        return []
    q_vec = embedder.encode([query], convert_to_numpy=True)
    scores, idx = index.search(q_vec.astype(np.float32), k)
    results = []
    for score, i in zip(scores[0], idx[0]):
        if i == -1:
            continue
        results.append((chunks[i], float(score)))
    return results


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

from groq import Groq
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("Groq client initialized.")


Groq client initialized.


In [ ]:
import os
from groq import Groq
import yfinance as yf

# 🔑 Set your Groq API key in Colab:
# os.environ["GROQ_API_KEY"] = "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"  # or use a .env

groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def groq_answer(question: str, context_chunks: list):
    """Ask Groq LLM using retrieved context chunks (RAG-style)."""
    context = "\n\n".join(context_chunks[:5])
    prompt = f"""
You are a financial analysis assistant.

Use ONLY the following context and answer the question.
If something is not in the context, say you cannot find it.

CONTEXT:
{context}

QUESTION:
{question}
"""
    completion = groq_client.chat.completions.create(
        model="llama3-8b-8192",   # choose your Groq model
        messages=[
            {"role": "system", "content": "You are a precise financial document assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    return completion.choices[0].message.content

def yahoo_verify(ticker: str):
    """Very simple Yahoo Finance retrieval for verification."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "symbol": ticker,
            "shortName": info.get("shortName"),
            "marketCap": info.get("marketCap"),
            "epsTrailing": info.get("trailingEps"),
            "currency": info.get("currency"),
        }
    except Exception as e:
        return {"error": str(e)}


In [ ]:
APP_PATH = f"{OUTPUT_DIR}/streamlit_app.py"

app_code = r"""
import streamlit as st
import os, io, pickle, re
import fitz, pdfplumber, pytesseract
from PIL import Image
import numpy as np
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
from groq import Groq
import yfinance as yf

# Paths inside Colab drive
OUTPUT_DIR = "/content/drive/MyDrive/FinancialNER_Output"
PKL_PATH   = f"{OUTPUT_DIR}/ner_pipeline.pkl"

# Load NER pipeline
with open(PKL_PATH, "rb") as f:
    ner_pipeline = pickle.load(f)

# Embedding model
EMB_MODEL_NAME = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL_NAME)

# Groq client (expects GROQ_API_KEY env var)
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

st.set_page_config(page_title="Financial Report Intelligence", layout="wide")

st.title("📘 Financial Report Intelligence Dashboard")
st.caption("NER + RAG + Groq LLM + Yahoo Finance")

# ------------ helpers ------------

def extract_pages(pdf_bytes: bytes):
    pages = []
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        for i, page in enumerate(doc):
            txt = page.get_text("text") or ""
            if len(txt.strip()) < 30:
                pix = page.get_pixmap(dpi=150)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                txt = pytesseract.image_to_string(img)
            pages.append({"page": i+1, "text": txt})
    return pages

def chunk_text(pages, max_chars=800, overlap=200):
    chunks, meta = [], []
    for p in pages:
        text = re.sub(r"\s+", " ", p["text"]).strip()
        start = 0
        while start < len(text):
            end = min(len(text), start + max_chars)
            chunk = text[start:end]
            chunks.append(chunk)
            meta.append({"page": p["page"]})
            start = end - overlap
            if start < 0:
                start = 0
            if end == len(text):
                break
    return chunks, meta

def build_index(chunks):
    if not chunks:
        return None, None
    vectors = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors.astype(np.float32))
    return index, vectors

def search_chunks(query, chunks, index, k=5):
    if index is None:
        return []
    q_vec = embedder.encode([query], convert_to_numpy=True)
    scores, idx = index.search(q_vec.astype(np.float32), k)
    results = []
    for score, i in zip(scores[0], idx[0]):
        if i == -1:
            continue
        results.append(chunks[i])
    return results

def groq_answer(question, context_chunks):
    context = "\\n\\n".join(context_chunks[:5])
    prompt = f\"\"\"Use ONLY the following context from a financial report
to answer the question. If the answer is not clearly present, say so.

CONTEXT:
{context}

QUESTION:
{question}
\"\"\"
    completion = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role":"system","content":"You are a precise financial analysis assistant."},
            {"role":"user","content":prompt}
        ],
        temperature=0.0,
    )
    return completion.choices[0].message.content

def yahoo_verify(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            "symbol": ticker,
            "shortName": info.get("shortName"),
            "marketCap": info.get("marketCap"),
            "epsTrailing": info.get("trailingEps"),
            "currency": info.get("currency"),
        }
    except Exception as e:
        return {"error": str(e)}

# ------------ UI ------------

uploaded = st.sidebar.file_uploader("📄 Upload Financial PDF", type=["pdf"])
ticker   = st.sidebar.text_input("Optional Yahoo Finance ticker (e.g. AAPL)")

if uploaded:
    raw = uploaded.read()
    with st.spinner("Extracting pages with OCR fallback..."):
        pages = extract_pages(raw)

    full_text = "\\n\\n".join(p["text"] for p in pages)
    st.subheader("📑 Document Preview")
    st.text_area("First 1500 characters:", full_text[:1500], height=200)

    # ---------- NER ----------
    st.subheader("🧠 Named Entity Recognition (Custom Financial NER)")
    ents = []
    for p in pages:
        res = ner_pipeline(p["text"])
        for r in res:
            ents.append({
                "page": p["page"],
                "entity_type": r["entity_group"],
                "text": r["word"].replace("##",""),
                "score": float(r["score"])
            })
    if ents:
        df = pd.DataFrame(ents)
        st.dataframe(df, height=300)
    else:
        st.info("No entities found by NER.")

    # ---------- RAG index ----------
    st.subheader("📚 RAG Index (Chunks + Embeddings)")
    chunks, meta = chunk_text(pages)
    index, _ = build_index(chunks)
    st.write(f"Indexed {len(chunks)} chunks for semantic search.")

    # ---------- QA with Groq ----------
    st.subheader("💬 Ask a Question about this Report (Groq RAG)")
    question = st.text_input("Your question:")
    if question:
        with st.spinner("Retrieving relevant context and querying Groq..."):
            ctx_chunks = search_chunks(question, chunks, index, k=5)
            answer = groq_answer(question, ctx_chunks)
        st.markdown("**Answer:**")
        st.write(answer)
        with st.expander("Show retrieved context"):
            for i, ch in enumerate(ctx_chunks, 1):
                st.markdown(f"**Chunk {i}:** {ch}")

    # ---------- Yahoo Finance ----------
    st.subheader("📈 Yahoo Finance Verification (Optional)")
    if ticker:
        info = yahoo_verify(ticker)
        st.json(info)
    else:
        st.caption("Enter a ticker in the sidebar to fetch Yahoo Finance data.")

else:
    st.info("Upload a PDF from the sidebar to start.")
"""

with open(APP_PATH, "w") as f:
    f.write(app_code)

print("✅ Streamlit app saved to:", APP_PATH)


✅ Streamlit app saved to: /content/drive/MyDrive/FinancialNER_Output/streamlit_app.py


In [ ]:
# Start Streamlit in background
!nohup streamlit run "$APP_PATH" --server.port 8501 > /content/streamlit.log 2>&1 &

# Expose with ngrok
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)

# (Optional) check first lines of log if something breaks
!sleep 5
!sed -n '1,60p' /content/streamlit.log


ERROR:pyngrok.process.ngrok:t=2025-11-20T12:54:43+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-20T12:54:43+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-20T12:54:43+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
!pip install --upgrade pip -q
!apt-get update -qq
!apt-get install -y -qq tesseract-ocr libtesseract-dev poppler-utils
!pip install --quiet streamlit PyMuPDF pdfplumber pytesseract sentence-transformers transformers faiss-cpu groq yfinance pandas pillow
!npm install -g localtunnel


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving app.py to app.py


In [ ]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
import os
os.makedirs("/content/app", exist_ok=True)

# Copy files into /content/app
os.system(f"cp {APP_PATH} /content/app/app.py")
os.system(f"cp {PKL_PATH} /content/app/ner_json.pkl")

print("Files inside /content/app:")
os.listdir("/content/app")


NameError: name 'APP_PATH' is not defined

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

print("Groq API key set successfully.")


Groq API key set successfully.


In [ ]:
%%writefile /content/app/app.py
import streamlit as st
import pandas as pd
import numpy as np
import fitz
import pdfplumber
import pytesseract
from PIL import Image
import re, io, pickle, os
import faiss
import yfinance as yf

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, pipeline
from groq import Groq

# ---------------- CONFIG ----------------
NER_PKL = "/content/app/ner_json.pkl"   # make sure correct model is here
EMB_MODEL_NAME = "all-MiniLM-L6-v2"

st.set_page_config(page_title="Financial Analyzer", layout="wide")
st.title("📘 Financial Document Intelligence System")

# ---------------- LOAD NER ----------------
@st.cache_resource
def load_ner_pipeline():
    with open(NER_PKL, "rb") as f:
        loaded = pickle.load(f)

    # already a pipeline
    if hasattr(loaded, "__call__") and hasattr(loaded, "task"):
        return loaded

    if isinstance(loaded, dict) and "model_state_dict" in loaded:
        state_dict = loaded["model_state_dict"]
        keys = list(state_dict.keys())
        first_key = keys[0] if keys else ""

        # detect backbone from key prefix
        if first_key.startswith("distilbert."):
            base_model_name = "distilbert-base-uncased"
        elif first_key.startswith("bert."):
            base_model_name = "dslim/bert-base-NER"
        else:
            base_model_name = "distilbert-base-uncased"

        # load config & tokenizer without overriding num_labels
        config = AutoConfig.from_pretrained(base_model_name)
        tokenizer = AutoTokenizer.from_pretrained(base_model_name)

        # attach custom label maps if present
        if "id2label" in loaded and "label2id" in loaded:
            config.id2label = loaded["id2label"]
            config.label2id = loaded["label2id"]

        model = AutoModelForTokenClassification.from_config(config)
        # load weights with strict=False to ignore any extra keys safely
        model.load_state_dict(state_dict, strict=False)

        return pipeline(
            "ner",
            model=model,
            tokenizer=tokenizer,
            aggregation_strategy="simple"
        )

    raise ValueError("Unsupported PKL format")

ner_pipeline = load_ner_pipeline()

# ---------------- EMBEDDINGS ----------------
embedder = SentenceTransformer(EMB_MODEL_NAME)

# ---------------- GROQ ----------------
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def groq_answer(question, chunks):
    context = "\n\n".join(chunks[:5])
    prompt = f"""Use ONLY the following context from a financial report
to answer the question. If the answer is not clearly present, say you
cannot find it.

CONTEXT:
{context}

QUESTION:
{question}
"""
    resp = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise financial assistant."},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message.content

# ---------------- HELPERS ----------------
def extract_pdf_pages(pdf_bytes):
    pages = []
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        for i, p in enumerate(doc):
            text = p.get_text("text") or ""
            if len(text.strip()) < 30:
                pix = p.get_pixmap(dpi=150)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                text = pytesseract.image_to_string(img)
            pages.append({"page": i+1, "text": text})
    return pages

def chunk_text(pages, max_chars=800, overlap=200):
    chunks = []
    for p in pages:
        clean = re.sub(r"\s+", " ", p["text"]).strip()
        start = 0
        while start < len(clean):
            end = min(len(clean), start + max_chars)
            chunks.append(clean[start:end])
            start = end - overlap
            if start < 0:
                start = 0
            if end == len(clean):
                break
    return chunks

def build_faiss(chunks):
    if not chunks:
        return None
    vecs = embedder.encode(chunks, convert_to_numpy=True)
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vecs.astype("float32"))
    return index

def search_chunks(query, chunks, index, k=5):
    if index is None:
        return []
    qv = embedder.encode([query], convert_to_numpy=True)
    scores, idx = index.search(qv.astype("float32"), k)
    return [chunks[i] for i in idx[0]]

def yahoo_lookup(ticker):
    try:
        info = yf.Ticker(ticker).info
        return {
            "Name": info.get("shortName"),
            "Market Cap": info.get("marketCap"),
            "EPS": info.get("trailingEps"),
            "Currency": info.get("currency"),
        }
    except Exception as e:
        return {"error": str(e)}

# ---------------- UI ----------------
uploaded = st.file_uploader("Upload Annual Report PDF", type=["pdf"])
ticker = st.sidebar.text_input("Yahoo Finance Ticker (optional):")

if uploaded:
    raw = uploaded.read()
    pages = extract_pdf_pages(raw)

    preview = "\n\n".join(p["text"] for p in pages)[:1500]
    st.subheader("📄 Extracted Text Preview")
    st.text_area("Preview", preview, height=200)

    # NER
    st.header("🧠 NER: Extracted Entities")
    ents = []
    for p in pages:
        res = ner_pipeline(p["text"])
        for r in res:
            ents.append({
                "page": p["page"],
                "entity": r["entity_group"],
                "word": r["word"].replace("##", ""),
                "score": float(r["score"])
            })
    if ents:
        st.dataframe(pd.DataFrame(ents))
    else:
        st.info("No entities detected.")

    # Tables
    st.header("📊 Financial Tables")
    financial_keywords = [
        "revenue", "net income", "profit", "assets",
        "liabilities", "cash flow", "equity"
    ]
    tables_found = []
    with pdfplumber.open(io.BytesIO(raw)) as pdf:
        for i, p in enumerate(pdf.pages):
            try:
                tables = p.extract_tables()
            except:
                continue
            for tb in tables:
                df = pd.DataFrame(tb).dropna(how="all").dropna(axis=1, how="all")
                if df.empty:
                    continue
                flat = " ".join(df.astype(str).values.flatten()).lower()
                if any(k in flat for k in financial_keywords):
                    tables_found.append((i+1, df))

    if tables_found:
        for pg, df in tables_found:
            st.subheader(f"Table from Page {pg}")
            st.dataframe(df.reset_index(drop=True))
    else:
        st.info("No financial tables detected.")

    # RAG + Groq
    st.header("💬 Ask a Question (RAG + Groq LLM)")
    chunks = chunk_text(pages)
    index = build_faiss(chunks)
    question = st.text_input("Your question about this report:")
    if question:
        ctx = search_chunks(question, chunks, index, k=5)
        answer = groq_answer(question, ctx)
        st.success(answer)
        with st.expander("Retrieved Context"):
            for i, c in enumerate(ctx, 1):
                st.markdown(f"**Chunk {i}:** {c}")

    # Yahoo Finance
    st.header("📈 Yahoo Finance Verification")
    if ticker:
        st.json(yahoo_lookup(ticker))
else:
    st.info("Upload a PDF to start.")


Overwriting /content/app/app.py


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


In [ ]:
!nohup streamlit run /content/app/app.py --server.port 8501 > /content/streamlit.log 2>&1 &


In [ ]:
# Check if streamlit is running and show last log lines
!ps aux | grep streamlit | grep -v grep || echo "no streamlit process"
print("----- streamlit.log (last 80 lines) -----")
!tail -n 80 /content/streamlit.log


root       16313 52.0  0.5 232464 70112 ?        S    18:40   0:01 /usr/bin/python3 /usr/local/bin/streamlit run /content/app/app.py --server.port 8501
----- streamlit.log (last 80 lines) -----



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.3.122:8501



In [ ]:
# Kill any previous streamlit
!pkill -f "streamlit" || echo "no old streamlit"

# Start streamlit again
!nohup streamlit run /content/app/app.py --server.port 8501 > /content/streamlit.log 2>&1 &

# Wait and print first log lines
import time, os
time.sleep(5)
print("----- streamlit.log (first 80 lines) -----")
!head -n 80 /content/streamlit.log


^C
----- streamlit.log (first 80 lines) -----



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.3.122:8501



In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate


2025-11-20T18:41:04Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-20T18:41:04Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-20T18:41:05Z INF +--------------------------------------------------------------------------------------------+
2025-11-20T18:41:05Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-20T18:41:05Z INF |  https://makes-polyester-travesti-mountain.trycloudfla

In [ ]:
import os

print("File exists:", os.path.exists("/content/app/ner_json.pkl"))
print("Size in bytes:", os.path.getsize("/content/app/ner_json.pkl") if os.path.exists("/content/app/ner_json.pkl") else "not found")

# Try reading raw bytes
try:
    with open("/content/app/ner_json.pkl", "rb") as f:
        data = f.read(200)
    print("First 200 bytes:", data)
except Exception as e:
    print("Error reading file:", e)


File exists: True
Size in bytes: 431425664
First 200 bytes: b'\x80\x04\x95\xad\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x10model_state_dict\x94\x8c\x0bcollections\x94\x8c\x0bOrderedDict\x94\x93\x94)R\x94(\x8c&bert.embeddings.word_embeddings.weight\x94\x8c\x0ctorch._utils\x94\x8c\x12_rebuild_tensor_v2\x94\x93\x94(\x8c\rtorch.storage\x94\x8c\x10_load_from_bytes\x94\x93\x94B\xf90O\x05\x80\x02\x8a\nl\xfc\x9cF\xf9 j'
